In [16]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

from gensim.models import Word2Vec

from sklearn.metrics.pairwise import cosine_similarity

In [17]:
filtered_music_log_df = pd.read_csv('filtered_user_music_log.csv')
music_info_df = pd.read_csv('music_info.csv')
music_total_data = pd.read_csv('music_total_with_genre.csv')

In [18]:
filtered_music_log_df_renamed = filtered_music_log_df.rename(columns={"TITLE": "Title", "ARTIST": "Artist"})

music_data = filtered_music_log_df_renamed.merge(
    music_info_df[['Title', 'Artist', 'Lyrics']],
    on=['Title', 'Artist'],
    how='left'
)

In [19]:
# 무작위로 100명의 사용자 ID를 선택
unique_users = music_data['USER_ID'].unique()
# selected_users = pd.Series(unique_users).sample(2182, random_state=1) # 재현성을 위해 random_state 고정

# 해당 사용자의 모든 청취 기록을 가져오기
selected_user_records = music_data[music_data['USER_ID'].isin(unique_users)]

In [20]:
# nltk의 불용어 사전을 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# 영어 불용어 리스트를 로딩
stop_words = set(stopwords.words('english'))

# 가사 전처리 함수 정의
def preprocess_lyrics(lyrics):
    # lyrics가 문자열인지 확인
    if not isinstance(lyrics, str):
        return []  # lyrics가 문자열이 아닐 경우 빈 리스트 반환

    # lyrics가 문자열일 경우 기존의 전처리를 계속 진행
    lyrics = lyrics.lower()
    lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(lyrics)
    filtered_lyrics = [word for word in word_tokens if not word in stop_words]
    return filtered_lyrics

# 전체 데이터셋에 대해 가사 전처리 수행
music_data['Processed_Lyrics'] = music_data['Lyrics'].apply(preprocess_lyrics)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# Word2Vec 모델을 훈련시키기 위한 가사 데이터셋 준비
lyrics_corpus = music_data['Processed_Lyrics'].tolist()

# Word2Vec 모델 초기화 및 훈련
# 차원 수를 100으로 설정하고, 주변 단어(window)를 3개로 설정, 최소 단어 빈도수(min_count)를 2로 설정
w2v_model = Word2Vec(sentences=lyrics_corpus, vector_size=100, window=3, min_count=2, workers=4)

In [28]:
# 사용자별 가사 프로필(평균 벡터)을 만드는 함수 정의
def create_user_lyrics_profile(user_id, user_records, w2v_model):
    # 선택된 사용자의 가사를 가져옵니다.
    user_lyrics = user_records[user_records['USER_ID'] == user_id]['Processed_Lyrics']

    # 사용자의 모든 가사에 대한 벡터를 초기화합니다.
    user_vector_sum = np.zeros(w2v_model.vector_size)
    word_count = 0

    # 사용자가 들은 각 가사에 대해
    for lyrics in user_lyrics:
        # 가사의 각 단어에 대해
        for word in lyrics:
            if word in w2v_model.wv:
                # 단어 벡터를 사용자 벡터에 추가
                user_vector_sum += w2v_model.wv[word]
                word_count += 1

    # 평균 가사 벡터를 계산합니다 (단어 카운트로 나누어줍니다).
    user_vector_avg = user_vector_sum / word_count if word_count else user_vector_sum

    return user_vector_avg

# 각 사용자별로 가사 프로필을 생성합니다.
user_profiles = {}

for user_id in unique_users:
    user_profiles[user_id] = create_user_lyrics_profile(user_id, music_data, w2v_model)

In [29]:
# Filter the user's listening history for the specific user ID ''
user_specific_log = filtered_music_log_df[filtered_music_log_df['USER_ID'] == '00daK56D']

# Calculate the play count for each genre for the specific user
user_specific_genre_counts = user_specific_log['GENRE'].value_counts()

# Get the top 3 genres for the specific user
user_specific_top_genres = user_specific_genre_counts.head(5).index.tolist()

# Filtering the music_total_with_genre dataframe for songs that match the user 'Nx2ySOiM' top genres
user_specific_top_genres_songs_df = music_total_data[music_total_data['genre'].isin(user_specific_top_genres)]

In [30]:
# 태그 데이터를 전처리하는 함수를 정의합니다.
def preprocess_tags(tag_string):
    # '#' 기호를 기준으로 태그를 분리합니다.
    tags = tag_string.strip().split('#')
    # 빈 문자열을 제거합니다.
    tags = [tag for tag in tags if tag]  # 공백 태그 제거
    return tags

# 태그 데이터에 전처리 함수를 적용합니다.
user_specific_top_genres_songs_df['Processed_Tags'] = user_specific_top_genres_songs_df['tag'].apply(preprocess_tags)

<ipython-input-30-36c1b3aaf4e1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_specific_top_genres_songs_df['Processed_Tags'] = user_specific_top_genres_songs_df['tag'].apply(preprocess_tags)


In [31]:
# 태그를 벡터로 변환하는 함수를 정의합니다.
def vectorize_tags(tags, w2v_model):
    tag_vectors = []
    for tag in tags:
        # 태그 내의 각 단어에 대해 벡터를 얻고 평균을 계산합니다.
        tag_word_vectors = [w2v_model.wv[word] for word in tag.split() if word in w2v_model.wv]
        if tag_word_vectors:  # 태그가 모델 단어장에 있는 경우에만 평균 벡터를 계산합니다.
            tag_vectors.append(np.mean(tag_word_vectors, axis=0))
    return np.mean(tag_vectors, axis=0) if tag_vectors else np.zeros(w2v_model.vector_size)

# 각 태그를 벡터로 변환합니다.
user_specific_top_genres_songs_df['Tag_Vector'] = user_specific_top_genres_songs_df['Processed_Tags'].apply(lambda tags: vectorize_tags(tags, w2v_model))

<ipython-input-31-aa34b130ecc8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_specific_top_genres_songs_df['Tag_Vector'] = user_specific_top_genres_songs_df['Processed_Tags'].apply(lambda tags: vectorize_tags(tags, w2v_model))


In [32]:
user_id_to_recommend = '00daK56D'

# 사용자 프로필 벡터와 모든 태그 벡터 사이의 코사인 유사도를 계산하고 상위 N개의 추천과 함께 유사도를 반환하는 함수
def recommend_songs_with_similarity(user_profile_vector, tag_vectors, songs_data, top_n=20):
    # 사용자 프로필 벡터를 코사인 유사도 계산을 위해 reshape
    user_vector_reshaped = user_profile_vector.reshape(1, -1)

    # 모든 태그 벡터와의 유사도 계산
    similarity_scores = cosine_similarity(user_vector_reshaped, tag_vectors)[0]

    # 유사도 점수를 기반으로 상위 N개의 인덱스를 가져옵니다
    top_indices = similarity_scores.argsort()[-top_n:][::-1]

    # 상위 N개의 노래 추천 정보와 유사도 점수를 함께 반환
    recommendations_with_scores = songs_data.iloc[top_indices]
    recommendations_with_scores['similarity'] = similarity_scores[top_indices]
    return recommendations_with_scores[['title', 'artist', 'tag', 'similarity']]

# 모든 태그 벡터를 하나의 배열로 추출합니다.
tag_vectors_matrix = np.array(list(user_specific_top_genres_songs_df['Tag_Vector']))

# 'UGqP66iE' 사용자 ID에 대한 노래 추천을 받고 유사도 점수를 포함하여 출력합니다.
user_profile_vector_for_similarity = user_profiles[user_id_to_recommend]  # 해당 사용자의 프로필 벡터를 가져옵니다.
recommendations_with_similarity = recommend_songs_with_similarity(user_profile_vector_for_similarity, tag_vectors_matrix, user_specific_top_genres_songs_df)
recommendations_with_similarity

<ipython-input-32-73a11518cf26>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations_with_scores['similarity'] = similarity_scores[top_indices]


,title,artist,tag,similarity
36592,거짓말,지오디(god),#2000년대#KPOP#가요#공연/라이브#공연/페스티벌#국내#그리움#눈오는 날#발라...,0.558487
36689,겨울,정준일,#KPOP#가요#겨울#공부할 때#국내#그리움#눈오는 날#발라드한#쌀쌀한 날#쓸쓸한#...,0.556651
52031,찢어주세요,박원,#감성적인#국내#국내 인디#그리움#밤/새벽#비/흐림#쌀쌀한 날#쓸쓸한#우울할때#울고...,0.554956
50461,이별하러 가는 길,임한별,#가요#감성적인#겨울#국내#국내 인디#그리움#노래방#눈오는 날#발라드한#밤/새벽#쌀...,0.552178
45832,사랑한 후에,박효신,#1990년대#2000년대#2010년대#8090세대#가요#가을#감성적인#겨울#공부할...,0.542189
47488,아니면서,성시경,#가요#겨울#국내#그리움#눈오는 날#발라드한#쌀쌀한 날#애절한#울고 싶을때#이별/슬...,0.532535
47845,안 될 사랑,한동근,#가요#겨울#국내#눈오는 날#발라드한#쌀쌀한 날#애절한#울고 싶을때#이별/슬픔#카페,0.532513
1661,Alone Again (Gilbert O'Sullivan),이소라,#감성적인#국내#국내 인디#쌀쌀한 날#인디,0.527240
51043,잘 해낼 거예요,성태(포스트맨),#목소리/음색#발라드한#쌀쌀한 날,0.527240
12755,Home Again,권순관,#감성적인#국내#국내 인디#쌀쌀한 날#인디,0.527240
